In [ ]:
import os
import dotenv

dotenv.load_dotenv()



In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [35]:
# Generate text
text = llm.invoke("Hello, how are you?")
text

AIMessage(content="I am an AI language model, so I don't have feelings or experiences like humans do. However, I am here to assist you with any questions or tasks you may have! How can I help you today? \n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-1977d8aa-4905-42fc-b9de-168f9262678b-0', usage_metadata={'input_tokens': 7, 'output_tokens': 44, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}})

In [36]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='resources/codebasics_faqs.csv', source_column='prompt')
data = loader.load()

In [37]:
# from langchain_community.embeddings import HuggingFaceInstructEmbeddings

# model_name = "hkunlp/instructor-large"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True}
# hf = HuggingFaceInstructEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )
# e = embeddings.embed_query('How to install python?')


In [38]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_encoder = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)
    

In [39]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(embedding=embedding_encoder, documents=data)

In [40]:
vector_store.save_local("resources/vector_store")

In [41]:
retriever = vector_store.as_retriever()

In [42]:
# retreiver.get_relevant_documents('For how long is this course valid')

retriever.invoke('For how long is this course valid')

[Document(metadata={'source': 'Once purchased, is this course available for lifetime access?', 'row': 22}, page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes'),
 Document(metadata={'source': 'What is the duration of this bootcamp? How long will it last?', 'row': 8}, page_content='prompt: What is the duration of this bootcamp? How long will it last?\nresponse: You can complete all courses in 3 months if you dedicate 2-3 hours per day.'),
 Document(metadata={'source': 'Does this bootcamp have lifetime access?', 'row': 7}, page_content='prompt: Does this bootcamp have lifetime access?\nresponse: Yes'),
 Document(metadata={'source': 'Will the course help me in PL - 300 Microsoft exam preparation?', 'row': 26}, page_content='prompt: Will the course help me in PL - 300 Microsoft exam preparation?\nresponse: Yes, this course will certainly help because we cover the majority of the skills measured in the PL - 300 exam in this course. However, ple

In [58]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)